### **Documentação:** Script de Amostragem e Análise (AurumDataSampler)

#### **1. Objetivo**

Este é um **script utilitário** de Análise Exploratória de Dados (EDA). Seu principal objetivo é ajudar o desenvolvedor a entender o ecossistema de dados do projeto Aurum, que está em constante evolução.

Ele **não coleta** novos dados. Em vez disso, ele **varre** o diretório `data/` em busca de todos os arquivos de dados `.parquet` já processados e, para cada um, executa as seguintes ações:
1.  **Descobre:** Lista todos os datasets `.parquet` disponíveis.
2.  **Amostra:** Carrega uma amostra pequena e inteligente dos dados (usando amostragem estratificada por data, se possível).
3.  **Analisa:** Gera um relatório básico de qualidade dos dados (contagem de linhas, colunas, tipos de dados, valores nulos, duplicatas e estatísticas numéricas).
4.  **Reporta:** Salva as amostras em arquivos `.csv` fáceis de visualizar e cria um relatório consolidado em `.json`.

Este script é fundamental para validar rapidamente a saída de cada etapa do pipeline (coleta de notícias, processamento de fundamentos, etc.).

#### **2. Configuração (Input)**

O script foi projetado para funcionar sem configuração manual. Ele assume a seguinte estrutura de diretório:

* **data/ (Diretório Raiz):** O script inicia sua busca aqui.
* **.parquet** (Arquivos Alvo):** Ele procurará recursivamente em **todos** os subdiretórios dentro de data/ por qualquer arquivo que termine com a extensão .parquet.

## 3. Saída (Output)

Ao executar main(), o script cria um novo diretório: data/samples/. Este diretório conterá:

1.  **Amostras em CSV (`*.sample.csv`):**
    * `raw_news_data_sample.csv`
    * `fundamentals_wide_sample.csv`
    * `aurum_quality_scores_complete_sample.csv`
    * *(...e um `.csv` para cada `.parquet` encontrado)*

2.  **Relatório Consolidado (`aurum_sampling_report.json`):**
    * Um único arquivo JSON que armazena os metadados e a análise de qualidade de todos os datasets processados. Este arquivo é ideal para monitoramento programático do "data health" (saúde dos dados) do projeto.

---

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import json

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class AurumDataSampler:
    """
    Classe para amostragem e análise exploratória dos dados do Aurum
    """
    
    def __init__(self, base_path: str = "data"):
        self.base_path = Path(base_path)
        self.sample_size = 100  # Tamanho padrão da amostra
        
    def list_available_datasets(self) -> Dict[str, Path]:
        """
        Lista todos os datasets Parquet disponíveis
        """
        datasets = {}
        
        # Procurar recursivamente por arquivos .parquet
        for parquet_file in self.base_path.rglob("*.parquet"):
            relative_path = parquet_file.relative_to(self.base_path)
            datasets[str(relative_path)] = parquet_file
            
        return datasets
    
    def load_dataset_sample(self, dataset_path: Path, sample_size: int = None, 
                          random_state: int = 42) -> pd.DataFrame:
        """
        Carrega uma amostra representativa do dataset
        """
        if sample_size is None:
            sample_size = self.sample_size
            
        logger.info(f"📊 Carregando amostra de {dataset_path}...")
        
        try:
            # Ler o dataset completo para obter metadados
            full_df = pd.read_parquet(dataset_path)
            total_rows = len(full_df)
            
            logger.info(f"   📈 Dataset completo: {total_rows} linhas, {len(full_df.columns)} colunas")
            
            # Estratégias de amostragem baseadas no tamanho do dataset
            if total_rows <= sample_size:
                # Dataset pequeno - retornar tudo
                sample_df = full_df.copy()
                logger.info("   🔍 Dataset pequeno - retornando dados completos")
            else:
                # Dataset grande - amostragem estratificada quando possível
                sample_df = self._stratified_sampling(full_df, sample_size, random_state)
                
            return sample_df
            
        except Exception as e:
            logger.error(f"❌ Erro ao carregar {dataset_path}: {e}")
            return pd.DataFrame()
    
    def _stratified_sampling(self, df: pd.DataFrame, sample_size: int, random_state: int) -> pd.DataFrame:
        """
        Amostragem estratificada tentando manter proporções de categorias importantes
        """
        sample_df = df.copy()
        
        # Tentar estratificar por ano se a coluna existir
        year_columns = [col for col in sample_df.columns if 'ANO' in col or 'YEAR' in col or 'DATA' in col]
        
        if year_columns:
            try:
                year_col = year_columns[0]
                # Amostragem estratificada por ano
                stratified_sample = sample_df.groupby(year_col, group_keys=False).apply(
                    lambda x: x.sample(n=min(len(x), max(1, sample_size // sample_df[year_col].nunique())), 
                                      random_state=random_state)
                )
                
                # Se a amostra estratificada for muito pequena, completar com amostra aleatória
                if len(stratified_sample) < sample_size:
                    remaining = sample_size - len(stratified_sample)
                    additional_sample = sample_df.drop(stratified_sample.index).sample(
                        n=remaining, random_state=random_state)
                    stratified_sample = pd.concat([stratified_sample, additional_sample])
                
                logger.info(f"   🎯 Amostragem estratificada por {year_col}: {len(stratified_sample)} linhas")
                return stratified_sample
                
            except Exception as e:
                logger.warning(f"   ⚠️ Amostragem estratificada falhou: {e}")
        
        # Fallback: amostra aleatória simples
        simple_sample = sample_df.sample(n=min(sample_size, len(sample_df)), random_state=random_state)
        logger.info(f"   🎲 Amostra aleatória simples: {len(simple_sample)} linhas")
        return simple_sample
    
    def generate_basic_report(self, dataset_path: Path, sample_size: int = 50) -> Dict:
        """
        Gera relatório básico do dataset (sem dados complexos para JSON)
        """
        # Carregar amostra
        sample_df = self.load_dataset_sample(dataset_path, sample_size)
        
        if sample_df.empty:
            return {
                'dataset_name': dataset_path.name,
                'dataset_path': str(dataset_path),
                'error': 'Falha ao carregar dataset'
            }
        
        # Estatísticas básicas
        basic_stats = {
            'total_rows_original': len(pd.read_parquet(dataset_path)),
            'sample_size': len(sample_df),
            'columns_count': len(sample_df.columns),
            'memory_usage_mb': round(sample_df.memory_usage(deep=True).sum() / 1024**2, 2),
            'columns_list': list(sample_df.columns),
            'dtypes': {col: str(dtype) for col, dtype in sample_df.dtypes.items()}
        }
        
        # Informações de data
        date_info = self._get_date_info(sample_df)
        
        # Qualidade básica dos dados
        quality_info = self._get_basic_quality_info(sample_df)
        
        report = {
            'dataset_name': dataset_path.name,
            'dataset_path': str(dataset_path),
            'basic_stats': basic_stats,
            'date_info': date_info,
            'quality_info': quality_info
        }
        
        return report
    
    def _get_date_info(self, df: pd.DataFrame) -> Dict:
        """Extrai informações de data de forma segura"""
        date_info = {}
        date_columns = df.select_dtypes(include=['datetime64']).columns
        
        for col in date_columns:
            try:
                if col in df.columns and not df[col].isna().all():
                    date_info[col] = {
                        'min': str(df[col].min()),
                        'max': str(df[col].max()),
                        'null_count': int(df[col].isna().sum()),
                        'null_percentage': round((df[col].isna().sum() / len(df)) * 100, 2)
                    }
            except Exception as e:
                date_info[col] = {'error': f'Erro ao processar: {str(e)}'}
        
        return date_info
    
    def _get_basic_quality_info(self, df: pd.DataFrame) -> Dict:
        """Analisa qualidade básica dos dados"""
        quality_info = {
            'total_rows': len(df),
            'complete_cases': len(df.dropna()),
            'duplicate_rows': int(df.duplicated().sum()),
            'columns_quality': {}
        }
        
        for column in df.columns:
            try:
                col_data = df[column]
                col_info = {
                    'dtype': str(col_data.dtype),
                    'null_count': int(col_data.isna().sum()),
                    'null_percentage': round((col_data.isna().sum() / len(df)) * 100, 2),
                    'unique_count': int(col_data.nunique())
                }
                
                # Estatísticas para colunas numéricas
                if pd.api.types.is_numeric_dtype(col_data):
                    col_info.update({
                        'mean': float(col_data.mean()) if not col_data.isna().all() else None,
                        'std': float(col_data.std()) if not col_data.isna().all() else None,
                        'min': float(col_data.min()) if not col_data.isna().all() else None,
                        'max': float(col_data.max()) if not col_data.isna().all() else None
                    })
                
                quality_info['columns_quality'][column] = col_info
                
            except Exception as e:
                quality_info['columns_quality'][column] = {'error': f'Erro na análise: {str(e)}'}
        
        return quality_info
    
    def create_simple_sampling_report(self, output_dir: str = "data/samples") -> Dict:
        """
        Cria relatório simplificado de amostragem para todos os datasets
        """
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        datasets = self.list_available_datasets()
        reports = {}
        
        logger.info("🔍 INICIANDO AMOSTRAGEM DOS DATASETS AURUM")
        logger.info(f"📁 Encontrados {len(datasets)} datasets:")
        
        for name, path in datasets.items():
            logger.info(f"   📊 {name}")
        
        print("\n" + "="*60)
        print("🎯 RELATÓRIO DE AMOSTRAGEM - AURUM DATA SAMPLES")
        print("="*60)
        
        for dataset_name, dataset_path in datasets.items():
            print(f"\n📁 DATASET: {dataset_name}")
            print("-" * 40)
            
            # Gerar relatório básico
            report = self.generate_basic_report(dataset_path)
            reports[dataset_name] = report
            
            # Carregar amostra para salvar como CSV
            sample_df = self.load_dataset_sample(dataset_path, 50)
            
            if not sample_df.empty:
                # Salvar amostra como CSV
                sample_csv_path = output_path / f"{dataset_path.stem}_sample.csv"
                sample_df.to_csv(sample_csv_path, index=False, encoding='utf-8')
                
                # Mostrar resumo no console
                basic_stats = report['basic_stats']
                print(f"📈 Estatísticas:")
                print(f"   • Linhas totais: {basic_stats['total_rows_original']:,}")
                print(f"   • Amostra: {basic_stats['sample_size']} linhas")
                print(f"   • Colunas: {basic_stats['columns_count']}")
                print(f"   • Uso de memória: {basic_stats['memory_usage_mb']:.2f} MB")
                
                # Mostrar primeiras linhas
                print(f"\n👀 Primeiras 3 linhas da amostra:")
                print(sample_df.head(3).to_string(index=False))
                
                print(f"\n💾 Amostra salva em: {sample_csv_path}")
            else:
                print("❌ Não foi possível carregar amostra deste dataset")
        
        # Salvar relatório consolidado (agora seguro para JSON)
        report_path = output_path / "aurum_sampling_report.json"
        
        try:
            with open(report_path, 'w', encoding='utf-8') as f:
                json.dump(reports, f, indent=2, ensure_ascii=False, default=str)
            logger.info(f"📋 Relatório consolidado salvo em: {report_path}")
        except Exception as e:
            logger.error(f"❌ Erro ao salvar relatório JSON: {e}")
        
        return reports

    def quick_preview(self, dataset_pattern: str = None, sample_size: int = 10):
        """
        Visualização rápida dos datasets
        """
        datasets = self.list_available_datasets()
        
        print("🚀 VISUALIZAÇÃO RÁPIDA DOS DATASETS AURUM")
        print("=" * 50)
        
        for name, path in datasets.items():
            if dataset_pattern and dataset_pattern.lower() not in name.lower():
                continue
                
            print(f"\n📊 {name}")
            print("-" * 40)
            
            sample_df = self.load_dataset_sample(path, sample_size)
            if not sample_df.empty:
                print(f"📋 Shape: {sample_df.shape}")
                print(f"🎯 Amostra de {len(sample_df)} linhas:")
                print(sample_df.to_string(index=False))
            else:
                print("❌ Não foi possível carregar amostra")
            
            print("\n" + "=" * 50)

# Funções de uso rápido
def quick_sample(dataset_pattern: str, sample_size: int = 15) -> pd.DataFrame:
    """
    Função rápida para obter amostra de datasets que correspondam ao padrão
    """
    sampler = AurumDataSampler()
    datasets = sampler.list_available_datasets()
    
    matching_datasets = []
    for name, path in datasets.items():
        if dataset_pattern.lower() in name.lower():
            matching_datasets.append((name, path))
    
    if not matching_datasets:
        available = "\n".join(datasets.keys())
        print(f"❌ Nenhum dataset encontrado com '{dataset_pattern}'.")
        print(f"📁 Datasets disponíveis:\n{available}")
        return pd.DataFrame()
    
    if len(matching_datasets) == 1:
        name, path = matching_datasets[0]
        sample = sampler.load_dataset_sample(path, sample_size)
        print(f"🎯 Amostra de {name} ({len(sample)} linhas):")
        print(sample.to_string(index=False))
        return sample
    else:
        print(f"🔍 Múltiplos datasets encontrados com '{dataset_pattern}':")
        for i, (name, path) in enumerate(matching_datasets, 1):
            print(f"   {i}. {name}")
        
        choice = input("👉 Escolha o número do dataset (ou Enter para o primeiro): ").strip()
        try:
            choice_idx = int(choice) - 1 if choice else 0
            name, path = matching_datasets[choice_idx]
            sample = sampler.load_dataset_sample(path, sample_size)
            print(f"🎯 Amostra de {name} ({len(sample)} linhas):")
            print(sample.to_string(index=False))
            return sample
        except (ValueError, IndexError):
            print("❌ Escolha inválida.")
            return pd.DataFrame()

def list_datasets():
    """Lista todos os datasets disponíveis"""
    sampler = AurumDataSampler()
    datasets = sampler.list_available_datasets()
    
    print("📁 DATASETS DISPONÍVEIS NO AURUM:")
    print("=" * 50)
    
    for i, (name, path) in enumerate(datasets.items(), 1):
        try:
            # Tentar obter informações básicas
            df_sample = pd.read_parquet(path, nrows=1)
            print(f"{i:2d}. {name}")
            print(f"    📊 Colunas: {len(df_sample.columns)}, Estilo: {df_sample.shape[1]}xN")
            print(f"    📍 {path}")
        except:
            print(f"{i:2d}. {name} (❌ erro ao carregar)")
    
    return datasets

# Execução principal
def main():
    """
    Executa amostragem completa de todos os datasets
    """
    sampler = AurumDataSampler()
    reports = sampler.create_simple_sampling_report()
    
    print("\n" + "="*60)
    print("✅ AMOSTRAGEM CONCLUÍDA!")
    print("="*60)
    
    # Resumo final
    total_datasets = len(reports)
    successful_samples = sum(1 for report in reports.values() if 'error' not in report)
    
    print(f"📊 Resumo Final:")
    print(f"   • Datasets processados: {total_datasets}")
    print(f"   • Amostras geradas com sucesso: {successful_samples}")
    print(f"   • Local das amostras: data/samples/")
    
    print(f"\n🎯 COMANDOS RÁPIDOS:")
    print(f"   • list_datasets() - Lista todos os datasets")
    print(f"   • quick_sample('quality') - Amostra de datasets de qualidade")
    print(f"   • quick_sample('news') - Amostra de datasets de notícias")
    print(f"   • quick_sample('fundamentals') - Amostra de dados fundamentalistas")

if __name__ == "__main__":
    main()

# Exemplos de uso após execução
print("\n" + "="*60)
print("🚀 EXEMPLOS DE USO RÁPIDO - EXECUTE APÓS O SCRIPT:")
print("="*60)
print("""
# Listar todos os datasets
list_datasets()

# Amostra rápida de datasets de qualidade
quick_sample('quality', 10)

# Amostra de dados fundamentalistas  
quick_sample('fundamental', 15)

# Amostra de notícias
quick_sample('news', 8)

# Visualização rápida de tudo
sampler = AurumDataSampler()
sampler.quick_preview()
""")